# Spectral Normalized Gaussian Process (SNGP) Classification

In [ ]:
import os
import tempfile

import torch
from lightning import Trainer
from lightning.pytorch import seed_everything
from lightning.pytorch.loggers import CSVLogger

from lightning_uq_box.datamodules import TwoMoonsDataModule
from lightning_uq_box.models.fc_resnet import FCResNet
from lightning_uq_box.uq_methods import SNGPClassification
from lightning_uq_box.viz_utils import (
    plot_predictions_classification,
    plot_training_metrics,
    plot_two_moons_data,
)

%load_ext autoreload
%autoreload 2

In [ ]:
seed_everything(2)
# temporary directory for saving
my_temp_dir = tempfile.mkdtemp()

## Datamodule

In [ ]:
dm = TwoMoonsDataModule(batch_size=128)

In [ ]:
# define data
X_train, y_train, X_test, y_test, test_grid_points = (
    dm.X_train,
    dm.y_train,
    dm.X_test,
    dm.y_test,
    dm.test_grid_points,
)

In [ ]:
fig = plot_two_moons_data(X_train, y_train, X_test, y_test)

## Model

In [ ]:
feature_extractor = FCResNet(input_dim=2, features=64, depth=4)

In [ ]:
sngp = SNGPClassification(
    feature_extractor=feature_extractor,
    loss_fn=torch.nn.CrossEntropyLoss(),
    num_targets=2,
)

## Trainer

In [ ]:
logger = CSVLogger(my_temp_dir)
trainer = Trainer(
    max_epochs=100,  # number of epochs we want to train
    logger=logger,  # log training metrics for later evaluation
    log_every_n_steps=1,
    enable_checkpointing=False,
    enable_progress_bar=True,
    default_root_dir=my_temp_dir,
)

In [ ]:
trainer.fit(sngp, dm)

In [ ]:
fig = plot_training_metrics(
    os.path.join(my_temp_dir, "lightning_logs"), ["train_loss", "trainAcc"]
)

# Predictions

We can plot the predictions for a grid of test points spanning the extent of the input data and visualize the decision boundaries and corresponding uncertainty.

In [ ]:
preds = sngp.predict_step(test_grid_points)

In [ ]:
fig = plot_predictions_classification(
    X_test,
    y_test,
    preds["pred"].argmax(-1),
    test_grid_points,
    preds["pred_uct"].cpu().numpy(),
)